# SIGN EXERCISE

This is one of the exercises for Thewebconf2023.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dglai/Thewebconf2023-Tutorial/blob/master/sign_exercise.ipynb) [![GitHub](https://img.shields.io/badge/-View%20on%20GitHub-181717?logo=github&logoColor=ffffff)](https://github.com/dglai/Thewebconf2023-Tutorial/blob/master/sign_exercise.ipynb)

In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
os.environ['DGLBACKEND'] = "pytorch"

# Uncomment below to install required packages. If the CUDA version is not 11.6,
# check the https://www.dgl.ai/pages/start.html to find the supported CUDA
# version and corresponding command to install DGL.
!pip install dgl -f https://data.dgl.ai/wheels/cu118/repo.html > /dev/null

try:
    import dgl
    installed = True
except ImportError:
    installed = False
print("DGL installed!" if installed else "DGL not found!")

# **Exercise**

Use DGL sparse API to implement the SIGN model defined as 
$$f(X, A, A^2, \bar{A}, \bar{A}^2) = σ_1(g(X, A, A^2, \bar{A}, \bar{A}^2)Ω)$$ 
$$g(X, A, A^2, \bar{A}, \bar{A}^2) = σ_2([XΘ_0, AXΘ_1, A^2XΘ_2, \bar{A}XΘ_3, \bar{A}^2XΘ_4])$$
in which $\bar{A} = \bar{D}^{-\frac{1}{2}}(A+I)\bar{D}^{-\frac{1}{2}}$, where $I$ is the identity matrix, and $\bar{D}$ is the diagonal node degree matrix of $(A+I)$.
$Θ_x$ and $Ω$ are all learnable dense weight matrix.

$σ_1$ and $σ_2$ are non-linear activations (e.g. relu).

In [ ]:
import dgl.data

class SIGN(nn.Module):
    def __init__(self, in_size, out_size):
        super(SIGN, self).__init__()
        ...

    def forward(self):
        ...

# Load Cora dataset for training.
dataset = dgl.data.CoraGraphDataset()
print("Number of categories:", dataset.num_classes)

g = dataset[0]
in_size = g.ndata["feat"].shape[1]
out_size = dataset.num_classes
model = SIGN(in_size, out_size)

Define the model and execute the block, such that you can use the following code to train your model.

In [ ]:
import dgl
import dgl.data
import dgl.function as fn
import dgl.sparse as dglsp
import torch
import torch.nn as nn
import torch.nn.functional as F

def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    A = dglsp.spmatrix(torch.stack(g.edges()))
    for e in range(100):
        # Forward
        logits = model(A, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the
        # training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test
        # accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            info = (
                "In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f})"
                + ", test acc: {:.3f} (best {:.3f})"
            )
            print(
                info.format(
                    e, loss, val_acc, best_val_acc, test_acc, best_test_acc
                )
            )

train(g, model)